<a href="https://colab.research.google.com/github/jsb58p/CS-431-Operating-Systems/blob/main/Banker's%20Algorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [136]:
# Data
Processes = ['p0', 'p1', 'p2', 'p3', 'p4']
Resources = ['A', 'B', 'C']
NumberOfProcesses = len(Processes)
NumberOfResources = len(Resources)
Alloc = ([
    [0, 1, 0],
    [2, 0, 0],
    [3, 0, 2],
    [2, 1, 1],
    [0, 0, 2]
])
Max = ([
    [7, 5, 3],
    [3, 2, 2],
    [9, 0, 2],
    [2, 2, 2],
    [4, 3, 3]
])
Avail = ([3, 3, 2])


In [137]:
# Need Matrix Calculation
def CalculateNeedMatrix(Allocation, MaxNeed):
  Need = []
  for i in range(NumberOfProcesses):
    row = []
    for j in range(NumberOfResources):
        value = Max[i][j] - Alloc[i][j]
        row.append(value)
    Need.append(row)
  return Need


In [138]:
# Safety Algorithm
def SafetyAlgorithm(Processes, Allocation, Need, Available):
  # Initialize
  Work = Available.copy()
  Finish = [False] * NumberOfProcesses
  SafeSequence = []

  # Loop until no more processes can be added to safe sequence
  while False in Finish:
    found = False

    # Try to find an unfinished process that can complete
    for i in range(NumberOfProcesses):
      if Finish[i] == False:
        # Check if this process can be allocated needed resource
        canAllocate = True

        for j in range(NumberOfResources):
          if Need[i][j] > Work[j]:
            canAllocate = False
            break

        # If process can complete
        if canAllocate == True:
          # Add to safe sequence
          SafeSequence.append(Processes[i])

          # Mark as finished
          Finish[i] = True

          # Release resources
          for j in range(NumberOfResources):
            Work[j] = Work[j] + Allocation[i][j]

          found = True
          break

    # If no process could be found, system is unsafe
    if found == False:
      return(False,[])

  # System is in a safe state
  return(True, SafeSequence)


In [139]:
# Main Banker's Algorithm
def BankersAlgorithm(Processes, Allocation, MaxNeed, Available):
  # Calculate need matrix
  Need = CalculateNeedMatrix(Allocation, MaxNeed)

  # Run the safety algorithm
  return SafetyAlgorithm(Processes, Allocation, Need, Available)


In [140]:
from copy import deepcopy
def ResourceRequestAlgorithm(Processes, Allocation, MaxNeed, Available, ProcessIdx, Request):
  Need = CalculateNeedMatrix(Allocation, MaxNeed)

  # Step 1: Check if request exceeds need
  for j in range(NumberOfResources):
    if Request[j] > Need[ProcessIdx][j]:
      return False #Request exceeds maximum claim

  # Step 2: Check if resources are available
  for j in range(NumberOfResources):
    if Request[j] > Available[j]:
      return False # Resources not available

  # Step 3: Pretend to allocate resources
  TempAllocation = deepcopy(Allocation)
  TempAvailable = Available.copy()

  for j in range(NumberOfResources):
    TempAllocation[ProcessIdx][j] += Request[j]
    TempAvailable[j] -= Request[j]

  TempNeed = CalculateNeedMatrix(TempAllocation, MaxNeed)

  # Step 4: Check if resulting state is safe
  IsSafe, _ = SafetyAlgorithm(Processes, TempAllocation, TempNeed, TempAvailable)

  return IsSafe


In [141]:
# Banker's Algorithm function returns a safe state and sequence
isSafe, safeSequence = BankersAlgorithm(Processes, Alloc, Max, Avail)

# Print the result
if isSafe:
    print("Safe sequence:", safeSequence)
else:
    print("System is in an unsafe state.")


Safe sequence: ['p1', 'p3', 'p0', 'p2', 'p4']
